### 검증 세트
- 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련세트를 나누는 것

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
# 1. 훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2, random_state=42)

In [12]:
# 1-2. 훈련세트를 검증 세트로 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [13]:
print(sub_input.shape, val_input.shape, test_input.shape) # 훈련세트 4157개, 검증세트 1040개, 테스트세트 1300개

(4157, 3) (1040, 3) (1300, 3)


In [11]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### 교차 검증
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복 후 이 점수를 평균하여 최종 검증 점수를 얻음

In [17]:
# cross_validate() 교차 검증 함수
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
display(scores)

{'fit_time': array([0.02547789, 0.0098114 , 0.01012063, 0.02315474, 0.0097816 ]),
 'score_time': array([0.00111508, 0.0009377 , 0.00113487, 0.00128889, 0.00105214]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [18]:
# 교차 검증의 최종 점수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [20]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # 교차 검증에서 폴드를 어떻게 나눌지 결정해줌
print(np.mean(scores['test_score']))

0.855300214703487


In [23]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행 방법
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


### 하이퍼파라미터 튜닝
- GridSearchCV 클래스로 하이퍼파라미터 탐색과 교차 검증을 한번에 수행

In [24]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [27]:
# 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [28]:
# 학습
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [29]:
# 훈련 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [30]:
# 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [31]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_에 저장
print(gs.cv_results_['mean_test_score']) # 5-fold의 테스트 평균값

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [34]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # np.argmax() 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [35]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001,0.0001), 'max_depth':range(5,20,1), 'min_samples_split':range(2,100,10)}

In [36]:
print(params)

{'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]), 'max_depth': range(5, 20), 'min_samples_split': range(2, 100, 10)}


In [37]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [38]:
print(gs.best_params_) # 최상의 매개변수 조합 확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [39]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수 확인

0.8683865773302731


### 랜덤 서치
- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체 전달

In [41]:
# 싸이파이(scipy): 수치 계산 전용 라이브러리
from scipy.stats import uniform, randint

In [44]:
# randint 정수
rgen = randint(0,10) # 0 ~ 10 사이의 숫자중에
rgen.rvs(10) # 10개를 뽑아낸다

array([5, 1, 5, 3, 3, 3, 6, 4, 6, 1])

In [45]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 88, 103, 104,  97,  92, 108, 115,  94,  99, 100]))

In [46]:
# uniform 실수
ugen = uniform(0,1)
ugen.rvs(10)

array([0.8397349 , 0.80428633, 0.11255598, 0.25618251, 0.97365101,
       0.87612956, 0.8512672 , 0.78110025, 0.92401305, 0.58800721])

In [47]:
# 탐색할 매개변수의 딕셔너리 생성
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth':randint(20,50), 'min_samples_split':randint(2,25), 'min_samples_leaf':randint(1,25)}

In [51]:
# 샘플링 횟수는 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd2fba07040>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd2fba071f0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd2fb9fab20>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd2fba07e20>},
                   random_state=42)

In [52]:
# 최적의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [54]:
# 최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [56]:
# 최적의 모델
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
